# 🧹 **Bài học: Làm sạch dữ liệu (Data Cleaning)**

### 🎯 **Mục tiêu học tập**
- Hiểu vai trò của bước **data cleaning** trong quy trình Data Science.
- Nhận biết các dạng dữ liệu bị lỗi, thiếu hoặc ngoại lệ.
- Thực hành các kỹ thuật xử lý **missing data**, **outlier**, và **chuẩn hóa dữ liệu**.
- Ứng dụng các hàm thống kê mô tả để hiểu rõ đặc trưng cơ bản của dữ liệu.

## 🧠 **1. Giới thiệu về Data Cleaning**

Trong quy trình **Data Science**, bước làm sạch dữ liệu chiếm đến **60-80% thời gian** của dự án. Dữ liệu thô (raw data) thường chứa lỗi, thiếu sót hoặc định dạng không thống nhất.

Ví dụ về các vấn đề thường gặp:
- Dữ liệu thiếu (`NaN`, `None`).
- Cột dư thừa hoặc vô nghĩa.
- Dữ liệu sai kiểu (ví dụ: số điện thoại lưu dạng chuỗi có ký tự đặc biệt).
- Ngoại lệ (outlier) gây sai lệch thống kê.

Mục tiêu của bước này là giúp **dữ liệu trở nên đáng tin cậy** trước khi phân tích hoặc huấn luyện mô hình.

In [ ]:
import pandas as pd

# Đọc file dữ liệu Customer Call List
df = pd.read_csv('./data/Customer-Call-List.csv')

# Xem trước dữ liệu
df.head()

## 🔍 **2. Khám phá và kiểm tra dữ liệu ban đầu**

Trước khi làm sạch, ta cần hiểu cấu trúc và chất lượng dữ liệu thông qua các hàm mô tả và thống kê cơ bản:

In [ ]:
# Kiểm tra kích thước và thông tin tổng quan
df.info()

Qua thông tin nhận được sẽ có cái nhìn tổng quan ban đầu về dữ liệu hiện có. Sau đó bạn tuần tự thực hiện quy trình phân tích thông tin để tìm ra những điểm bất thường của dữ liệu.
> Lưu ý: Với mỗi trường hợp đặc thù thì thì dữ liệu sẽ có điểm bất thường riêng của nó.

1. Tìm dữ liệu rỗng `null`

In [ ]:
# Kiểm tra số lượng giá trị trống (missing values)
df.isnull().sum()

Ví dụ kiểm tra cột `Do_Not_Contact`  có dữ liệu trống không ?

In [ ]:
print("--- Các dòng có giá trị NaN trong 'Do_Not_Contact' ---")
missing_contacts = df[df['Do_Not_Contact'].isnull()]
print(missing_contacts)

Nếu có thì thống kế xem số lượng bao nhiêu để lên phương án xử lý

In [ ]:
# Để kiểm tra nhanh số lượng:
print(df['Do_Not_Contact'].isnull().sum())

2. Tìm Giá trị Bất thường về Định dạng/Loại (Non-Numeric Data)

In [ ]:
# Để kiểm tra nhanh số lượng:
print(df['Do_Not_Contact'].isna().sum())

3. Tìm Giá trị Ngoại lai (`Outliers`)

In [ ]:
import numpy as np

# 1. Xem xét các giá trị duy nhất (unique values)
print("--- Các giá trị Do_Not_Contact duy nhất ---")
print(df['Do_Not_Contact'].unique())

In [ ]:
# 2. Phân tích thống kê mô tả
print("\n--- Thống kê mô tả của 'Do_Not_Contact' ---")
print(df['Do_Not_Contact'].describe())

Do dataset này không có giá trị ngoại lại nên ví sau giả lập tình huống tạo ra có.

In [ ]:
# Tạo thêm cột giả định về thời lượng cuộc gọi (phút)
np.random.seed(42)
df['Call_Duration'] = np.random.normal(10, 3, len(df)).round(2)

# Thêm vài ngoại lệ thủ công
df.loc[0, 'Call_Duration'] = 60
df.loc[1, 'Call_Duration'] = 0.5

# Phát hiện outlier bằng IQR
Q1 = df['Call_Duration'].quantile(0.25)
Q3 = df['Call_Duration'].quantile(0.75)
IQR = Q3 - Q1
outliers = df[(df['Call_Duration'] < Q1 - 1.5*IQR) | (df['Call_Duration'] > Q3 + 1.5*IQR)]
outliers

4. Kiểm tra Giá trị Không hợp lệ về Logic (`Invalid Data`)

Giá trị **Không hợp lệ về Logic** (còn gọi là **dữ liệu không hợp logic** hoặc **Invalid Data** trong ngữ cảnh lỗi dữ liệu) là những giá trị tồn tại trong tập dữ liệu nhưng **không tuân theo các quy tắc hoặc mối quan hệ hợp lý** mà dữ liệu đó được kỳ vọng phải có.

Nó khác với giá trị **thiếu** (như `NULL` hoặc ô trống) hay giá trị **sai định dạng** (như nhập chữ vào ô số).

---

💡 Các dạng phổ biến của Lỗi Logic

Lỗi logic thường xảy ra khi giá trị nằm ngoài phạm vi chấp nhận được hoặc mâu thuẫn với các giá trị khác:

* **Giá trị ngoài phạm vi:**
    * **Ví dụ:** Tuổi của một người là **200** hoặc **-5**. (Về mặt kỹ thuật, 200 là một số hợp lệ, nhưng không hợp lý với tuổi người).
    * **Ví dụ:** Ngày sinh lớn hơn ngày hiện tại.

* **Mâu thuẫn giữa các trường:**
    * **Ví dụ:** Một người được ghi là **Học sinh** nhưng lại có chức danh là **Giám đốc**.
    * **Ví dụ:** Ngày bắt đầu công việc sau ngày kết thúc công việc.

* **Giá trị không phù hợp với kiểu dữ liệu đã được làm sạch:**
    * **Ví dụ:** Trong cột **`Paying Customer`** của file bạn tải lên, giá trị hợp lệ được mong đợi là **'Yes'** hoặc **'No'**. Tuy nhiên, có các giá trị biến thể như **'Y'** và **'N'**; nếu logic hệ thống chỉ chấp nhận đầy đủ 'Yes'/'No', thì 'Y'/'N' được coi là không hợp lệ về logic (hoặc cần phải được chuẩn hóa).
    * **Ví dụ:** Số điện thoại có định dạng không thống nhất, chẳng hạn như dùng dấu gạch ngang (`-`), gạch chéo (`/`), hoặc dấu sổ dọc (`|`) trong cột **`Phone_Number`** của file, nếu hệ thống chỉ chấp nhận một định dạng duy nhất.

---

🛠️ Cách nhận biết trong file của bạn

Dựa trên tệp **`Customer-Call-List.csv`** của bạn, đây là một số ví dụ có thể được coi là giá trị không hợp lệ về logic (hoặc cần chuẩn hóa):

| Cột | Giá trị có vấn đề | Lý do (Logic) |
| :--- | :--- | :--- |
| **`Phone_Number`** | `123/643/9775`, `876|678|3469`, `7066950392` | Giá trị có định dạng khác nhau (dùng `/`, `|`, hoặc không có dấu phân cách) trong khi định dạng chuẩn có thể là `XXX-XXX-XXXX`. |
| **`Paying Customer`** | `Y`, `N` | Các giá trị này là viết tắt, có thể không hợp logic nếu quy tắc hệ thống chỉ chấp nhận `Yes` hoặc `No`. |
| **`Last_Name`** | `/White`, trống (cho Gandalf) | Ký tự `/` trong tên hoặc tên trống có thể không hợp logic trong một số ứng dụng. |
| **`Do_Not_Contact`** | `Y`, trống (cho Gandalf) | Tương tự, nếu logic hệ thống chỉ chấp nhận `Yes` hoặc `No`. |
| **`Not_Useful_Column`** | `TRUE`, `FALSE` | Nếu cột này được cho là không hữu ích, nhưng lại chứa các giá trị boolean, có thể cần được xóa hoặc kiểm tra lại mục đích. |

## 🧩 **3. Xử lý dữ liệu thiếu (Missing Data)**

Các giá trị bị thiếu có thể ảnh hưởng đến việc thống kê và phân tích. Có 3 hướng xử lý phổ biến:

1. **Xoá dòng hoặc cột chứa nhiều giá trị trống.**  
2. **Thay thế giá trị thiếu bằng giá trị trung bình / trung vị / mode.**  
3. **Dự đoán hoặc nội suy giá trị bị thiếu (interpolation).**

👉 Trong ví dụ này, ta sẽ xoá các cột hoàn toàn trống và điền giá trị mặc định cho các ô trống quan trọng.

### 3.1 Xoá các cột không có dữ liệu

Trong quá trình làm sạch dữ liệu, ta thường cần **loại bỏ** (drop) một số **dòng hoặc cột không cần thiết**

Hàm `df.drop()` được sử dụng để **xóa hàng (rows)** hoặc **cột (columns)** khỏi DataFrame.

In [ ]:
# Sử dụng phương thức drop()
df = df.drop(columns=['Unnamed: 8', 'Unnamed: 9'])

### 3.2 Điền giá trị mặc định cho các cột bị thiếu

Khi làm việc với dữ liệu thực tế, ta thường gặp các ô bị thiếu giá trị (`NaN`, `None`, hoặc trống).
Nếu không xử lý, các giá trị này có thể gây lỗi trong tính toán hoặc làm sai lệch kết quả phân tích.

Hàm **`fillna()`** trong Pandas được dùng để **điền (thay thế)** các giá trị bị thiếu bằng một giá trị khác.



🔹 Cú pháp cơ bản

```python
DataFrame.fillna(value=None, method=None, inplace=False)
```

| Tham số   | Ý nghĩa                                                                                                               |
| --------- | --------------------------------------------------------------------------------------------------------------------- |
| `value`   | Giá trị được dùng để điền vào các ô bị thiếu (có thể là số, chuỗi hoặc từ điển `{column: value}`)                     |
| `method`  | Cách điền dữ liệu tự động, có thể là `'ffill'` (điền theo giá trị trước đó) hoặc `'bfill'` (điền theo giá trị sau đó) |
| `inplace` | Nếu `True`, thay đổi trực tiếp DataFrame gốc mà không cần gán lại biến mới                                            |

---

In [ ]:
# Điền giá trị mặc định cho các cột bị thiếu
df['Last_Name'].fillna('Unknown', inplace=True)
df['Phone_Number'].fillna('Missing', inplace=True)
df['Do_Not_Contact'].fillna('No Info', inplace=True)

In [ ]:
# Review lại dữ liệu
df.head()

💡 Ghi nhớ

* Dùng `fillna()` khi bạn **không muốn mất dữ liệu** do xoá dòng (`dropna()`).
* Kiểm tra trước khi điền bằng `df.isnull().sum()`.
* Thường dùng để **làm sạch dữ liệu thiếu (missing data)** trong bước *Data Cleaning*.
* Có thể áp dụng theo từng cột, toàn bộ DataFrame, hoặc dựa vào thống kê như mean/median/mode.


## 🧱 **4. Xử lý dữ liệu không hợp lệ và chuẩn hóa định dạng**

Một số giá trị có thể được nhập sai hoặc không thống nhất, ví dụ:
- Cột `Phone_Number` có các ký tự `/`, `|`, `-`.
- Cột `Paying Customer` có giá trị `Yes`, `No`, `Y`, `N`.
- Cột `Do_Not_Contact` có `Y`, `Yes`, `No`, `N`, `NaN`.

Chúng ta cần chuẩn hóa lại các giá trị này để phân tích dễ dàng hơn.

In [ ]:
# Làm sạch định dạng số điện thoại
df['Phone_Number'] = df['Phone_Number'].str.replace(r'[^0-9]', '', regex=True)
# Kiểm tra lại dữ liệu sau khi chuẩn hóa
df.head()

In [ ]:
# Chuẩn hoá cột Paying Customer
df['Paying Customer'] = df['Paying Customer'].replace({'Y': 'Yes', 'N': 'No'})
# Kiểm tra lại dữ liệu sau khi chuẩn hóa
df.head()

In [ ]:
# Chuẩn hoá cột Do_Not_Contact
df['Do_Not_Contact'] = df['Do_Not_Contact'].replace({'Y': 'Yes', 'N': 'No'})

# Kiểm tra lại dữ liệu sau khi chuẩn hóa
df.head()

## ⚠️ **5. Xử lý dữ liệu ngoại lệ (Outliers)**

Ngoại lệ (outlier) là các giá trị khác biệt bất thường so với phần còn lại. Ví dụ, nếu ta có cột `Age` hoặc `Income`, giá trị cực lớn hoặc cực nhỏ có thể ảnh hưởng tới trung bình.

👉 Dữ liệu hiện tại không có biến số học, nên ta có thể minh họa bằng cách tạo thêm cột `Call_Duration` để ví dụ xử lý ngoại lệ.

Sử dụng **quy tắc IQR (Interquartile Range)**:

In [ ]:
import numpy as np

# Tạo thêm cột giả định về thời lượng cuộc gọi (phút)
np.random.seed(42)
df['Call_Duration'] = np.random.normal(10, 3, len(df)).round(2)

# Thêm vài ngoại lệ thủ công
df.loc[0, 'Call_Duration'] = 60
df.loc[1, 'Call_Duration'] = 0.5

# Phát hiện outlier bằng IQR
Q1 = df['Call_Duration'].quantile(0.25)
Q3 = df['Call_Duration'].quantile(0.75)
IQR = Q3 - Q1
outliers = df[(df['Call_Duration'] < Q1 - 1.5*IQR) | (df['Call_Duration'] > Q3 + 1.5*IQR)]
outliers

## 📊 **6. Phân tích thống kê mô tả**

Các hàm thống kê giúp chúng ta hiểu rõ hơn về đặc điểm dữ liệu sau khi làm sạch:
- `df.describe()` : Thống kê cơ bản cho cột số.
- `df['col'].value_counts()` : Đếm tần suất xuất hiện.
- `df.corr()` : Phân tích tương quan (nếu có biến số học).

In [ ]:
# Thống kê mô tả cơ bản
df.describe()

# Đếm tần suất Paying Customer
df['Paying Customer'].value_counts()

Các hàm thống kê cơ bản

## ✅ **7. Kết luận**

Qua bài học này, chúng ta đã:
- Hiểu được **ý nghĩa và tầm quan trọng** của Data Cleaning.
- Biết cách xử lý **missing values**, **chuẩn hoá dữ liệu**, và **loại bỏ outliers**.
- Áp dụng các **phép thống kê mô tả** để hiểu cấu trúc dữ liệu sau làm sạch.

Bước tiếp theo trong quy trình **Data Science** sẽ là **Exploratory Data Analysis (EDA)** — trực quan hoá dữ liệu để tìm ra các mối quan hệ tiềm ẩn.

## 🧪 **8. Thực hành: Phân tích mô tả dữ liệu**